# Paramètres <br/>
$D$ la demande du client <br/>
$PPA_t$ la production du PPA (supposée gratuite) <br/>
$p_t$ le prix de l'éléricité sur le marché <br/>
$e_{bat}$ l'éfficité de la batterie <br/>
$c_{bat}$ la capacité de la batterie <br/>
$f_{bat}$ le flux maximal de la batterie <br/>
$e_{elec}$ l'éfficité de l'électrolyseur <br/>
$c_{elec}$ la capacité de l'électrolyseur <br/>
$c_{tank}$ la capacité du réservoir <br/>

# Variables <br/>
$\text{charge}_t$ la charge de la batterie à l'heure $t$ (en MWh)<br/>
$\text{prod}_t$ la production de l'électrolyseur à l'heure $t$ (en kg $H_2$)<br/>
$\text{stock}_t$ le stock d'hydrogène dans le reservoir à l'heure $t$ (en kg $H_2$)<br/>
$\text{elec}_t$ la consommation d'élécricité venant de la grille à l'heure $t$ (en MWh) <br/>

# Contraintes <br/>

Satisfaction de la demande du client : <br/>
$\forall t \in [|1, T|], \text{prod}_t + \text{stock}_t = D_t + \text{stock}_{t+1}$ <br/>
Consomation d'élécricité : <br/>
$\forall t \in [|1, T|], \text{elec}_t + \text{PPA}_t + e_{bat} \times \text{charge}_t = e_{elec} \times \text{prod}_t + \text{charge}_{t+1}$ <br/>
Contraintes de capacité : <br/>
$\forall t \in [|1, T|], \text{charge}_t \leq c_{bat}$ <br/>
$\forall t \in [|1, T|], \text{prod}_t \times e_{elec} \leq c_{elec} $ <br/>
$\forall t \in [|1, T|], \text{stock}_t \leq c_{tank} $ <br/>
Contraintes de flux : <br/>
$\forall t \in [|1, T|], |\text{charge}_t - \text{charge}_{t+1}| \leq f_{bat} $ <br/>

# Objectif <br/>

Minimiser le coût de l'élécricité : <br/>
$\min \sum_{t=1}^{T} p_t \times \text{elec}_t$

In [1]:
from pulp import *

In [ ]:
D = [1000]*24
PPA = [50] * 18 + [40, 30, 20, 10, 0, 50]
Cost_market = [20.79, 17.41, 16.24, 11.9, 9.77, 15.88, 24.88, 29.7, 35.01, 33.95, 29.9, 29.03]
Cost_market += [27.07, 26.43, 27.53, 29.05, 31.42, 39.92, 41.3, 41.51, 39.75, 30.13, 30.36, 32.4]
CostPPA = 20
capex_elec = 1200000 * 0.0004 #(euros/MW)
capex_gas_tank = 407
capex_battery =  250000 * 0.0002 #actually there are two different cases to deal with
Cap_max_elec = 1000 #(MW)
Cap_max_bat_flux = 100
Cap_max_bat_stock = 400
Cap_max_tank_stock = 500
efficiency_elec = 0.05 # (Mwh /kg H2)
efficiency_bat = 0.9

Ener_stored_bat_initial = 0
Hydrogen_stored_tank_initial = 0


In [ ]:
probleme = LpProblem("Cost_minimization", LpMinimize)

Ener_PPA = [LpVariable(f"Ener_PPA{i}", cat="Integer") for i in range(0, len(D) )]
Ener_Market = [LpVariable(f"Ener_Market{i}", cat="Integer") for i in range(0, len(D))]
Ener_Bat_flux = [LpVariable(f"Ener_Bat_flux{i}", cat="Integer") for i in range(0, len(D))]
Ener_used_by_elec = [LpVariable(f"Ener_Bat_flux{i}", cat="Integer") for i in range(0, len(D))]
Ener_stored_bat = [LpVariable(f"Ener_Bat_flux{i}", cat="Integer") for i in range(0, len(D))]
Hydrogen_stored_tank = [LpVariable(f"Ener_Bat_flux{i}", cat="Integer") for i in range(0, len(D))]

# Ener_stored_bat[0].setInitialValue(Ener_stored_bat_initial)
# Hydrogen_stored_tank[0].setInitialValue(Hydrogen_stored_tank_initial)

def cost(Ener_PPA, Ener_Market, Ener_used_by_elec, Ener_stored_bat, Hydrogen_stored_tank):
    Cost_energy =  [CostPPA*Ener_PPA[i] + Cost_market[i]*Ener_Market[i] + capex_elec*Ener_used_by_elec[i] for i in range(0, len(D))]
    Cost_storage = [capex_battery*Ener_stored_bat[i] + capex_gas_tank*Hydrogen_stored_tank[i] for i in range(0, len(D))]
    objective_function = sum(Cost_energy[i] + Cost_storage[i]  for i in range(0, len(D)))
    return objective_function

# Cost_energy =  [CostPPA*Ener_PPA[i] + Cost_market[i]*Ener_Market[i] + capex_elec*Ener_used_by_elec[i] for i in range(0, len(D))]
# Cost_storage = [capex_battery*Ener_stored_bat[i] + capex_gas_tank*Hydrogen_stored_tank[i] for i in range(0, len(D))]
# objective_function = sum(Cost_energy[i] + Cost_storage[i]  for i in range(0, len(D)))
probleme += cost(Ener_PPA, Ener_Market, Ener_used_by_elec, Ener_stored_bat, Hydrogen_stored_tank)

for i in range (0,len(D)):
    probleme += D[i]*efficiency_elec  <= Ener_used_by_elec[i]+ Hydrogen_stored_tank[i]*efficiency_elec 
    probleme += Hydrogen_stored_tank[i]*efficiency_elec == Ener_used_by_elec[i] - D[i]*efficiency_elec 
    probleme += Ener_PPA[i] <= PPA[i]
    probleme += Ener_Bat_flux[i] <= Ener_stored_bat[i]
    probleme += Ener_Bat_flux[i] <= Cap_max_bat_flux
    probleme += Ener_used_by_elec[i] <= Cap_max_elec
    probleme += Ener_used_by_elec[i]*efficiency_bat <= Ener_PPA[i]*efficiency_bat + Ener_Market[i]*efficiency_bat + Ener_Bat_flux[i]
    probleme += Ener_stored_bat[i]*efficiency_bat == Ener_PPA[i]*efficiency_bat + Ener_Market[i]*efficiency_bat + Ener_Bat_flux[i]- Ener_used_by_elec[i]*efficiency_bat
    probleme += Ener_stored_bat[i] <= Cap_max_bat_stock


probleme.solve()


PulpSolverError: Pulp: Error while executing c:\Users\33782\OneDrive\Documents\ProjetIMI-AirLiquide\lib\site-packages\pulp\solverdir\cbc\win\64\cbc.exe